In [ ]:
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV



In [ ]:

train_files = ['tile00.csv', 'tile01.csv', 'tile03.csv']
train = pd.concat([pd.read_csv(f) for f in train_files ], ignore_index=True)
X_train = train.loc[:, train.columns != 'class']
y_train = train[['class']]

X_train = X_train.rename(columns={x:y for x,y in zip(X_train.columns,range(0,len(X_train.columns)))})
X_train.fillna(0, inplace=True)
y_train.fillna(0, inplace=True)
y_train = y_train.values.ravel()

test = pd.read_csv('tile02.csv')
X_test = test.loc[:, test.columns != 'class']
y_test = test[['class']]
y_test = y_test.values.ravel()


In [ ]:
clf = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=30, random_state=0, verbose=1).fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:

bc_params = {"base_estimator__max_depth": [3,5,10,20],
          "base_estimator__max_features": [None, "auto"],
          "base_estimator__min_samples_leaf": [1, 3, 5, 7, 10],
          "base_estimator__min_samples_split": [2, 5, 7],
          'bootstrap_features': [False, True],
          'max_features': [0.5, 0.7, 1.0],
          'max_samples': [0.5, 0.7, 1.0],
          'n_estimators': [25, 50, 100],
}


bc_gs = GridSearchCV(BaggingClassifier(DecisionTreeClassifier()), bc_params, cv=5, verbose=1)